In [1]:
# Load libraries
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import pandas as pd

def load_and_tag_animals(animal_list, base_path="./DATA/VIEW/"):
    dataframes = []
    for animal in animal_list:
        df = pd.read_csv(f"./DATA/{animal}_df_view.csv")
        df["animal"] = animal
        dataframes.append(df)
    
    # Concatenate all dataframes
    all_animals = pd.concat(dataframes, ignore_index=True)
    return all_animals

In [3]:
# List of animals
animal_list = ["dog", "monkey", "horse", "rabbit", "pigs", "lion", "elephant", "bull", "cat"]

In [4]:
all = load_and_tag_animals(animal_list)
all.head()

,objectID,isHighlight,accessionNumber,accessionYear,isPublicDomain,primaryImage,primaryImageSmall,additionalImages,constituents,department,...,linkResource,metadataDate,repository,objectURL,tags,objectWikidata_URL,isTimelineWork,GalleryNumber,message,animal
0,544519.0,True,40.2.1,1940.0,True,https://images.metmuseum.org/CRDImages/eg/orig...,https://images.metmuseum.org/CRDImages/eg/web-...,['https://images.metmuseum.org/CRDImages/eg/or...,NaN,Egyptian Art,...,NaN,2023-09-16T04:54:53.55Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,"[{'term': 'Dogs', 'AAT_URL': 'http://vocab.get...",https://www.wikidata.org/wiki/Q29385810,False,119.0,NaN,dog
1,555715.0,False,24.1.51,1924.0,True,https://images.metmuseum.org/CRDImages/eg/orig...,https://images.metmuseum.org/CRDImages/eg/web-...,[],NaN,Egyptian Art,...,NaN,2023-09-16T04:54:53.55Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,"[{'term': 'Dogs', 'AAT_URL': 'http://vocab.get...",https://www.wikidata.org/wiki/Q116243597,False,109.0,NaN,dog
2,8807.0,False,43.75,1943.0,True,https://images.metmuseum.org/CRDImages/ad/orig...,https://images.metmuseum.org/CRDImages/ad/web-...,['https://images.metmuseum.org/CRDImages/ad/or...,"[{'constituentID': 166048, 'role': 'Maker', 'n...",The American Wing,...,NaN,2023-02-07T04:46:51.34Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,"[{'term': 'Dogs', 'AAT_URL': 'http://vocab.get...",https://www.wikidata.org/wiki/Q116337029,False,774.0,NaN,dog
3,559975.0,False,30.8.355,1930.0,True,https://images.metmuseum.org/CRDImages/eg/orig...,https://images.metmuseum.org/CRDImages/eg/web-...,[],NaN,Egyptian Art,...,NaN,2023-02-07T04:46:51.34Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,"[{'term': 'Dogs', 'AAT_URL': 'http://vocab.get...",https://www.wikidata.org/wiki/Q116276776,False,134.0,NaN,dog
4,545210.0,False,10.130.2520,1910.0,True,https://images.metmuseum.org/CRDImages/eg/orig...,https://images.metmuseum.org/CRDImages/eg/web-...,['https://images.metmuseum.org/CRDImages/eg/or...,NaN,Egyptian Art,...,NaN,2023-09-16T04:54:53.55Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,"[{'term': 'Dogs', 'AAT_URL': 'http://vocab.get...",https://www.wikidata.org/wiki/Q116244861,False,117.0,NaN,dog


## Chart 1: No. of artworks in each category

### Also tabled by "isHighlight" to find how many artworks in each category are famous

In [5]:
animal_total = all["animal"].value_counts(dropna=False).sort_index().reset_index()

chart1 = all.groupby('animal')['isHighlight'].value_counts(dropna=False).sort_index().reset_index()
# chart1
# chart1.to_csv("./DATA/CHARTS/chart1.csv", index = False)

## Chart 2: Department where each animal can be found the most

In [10]:
# Group by department
chart2 = all.groupby('animal')['department'].value_counts(dropna=False).sort_index().reset_index()

# Remove NAs
chart2 = chart2.dropna(subset=['department'])

# Combine both the European categories
# Filter rows where department starts with 'European'
european_data = chart2[chart2['department'].str.startswith('European')]

# Sum the values for 'count' in European rows
european_sum = european_data.groupby('animal')['count'].sum().reset_index()
european_sum['department'] = 'European'

# Remove the individual European rows from the original dataset
chart2 = chart2[~chart2['department'].str.startswith('European')]

# Concatenate the European sum with the rest of the dataset
chart2 = pd.concat([chart2, european_sum], ignore_index=True)

#chart2.to_csv("./DATA/CHARTS/chart2.csv")

## Chart 3: MEDIUM

### Since this analysis requires bucketing and classification, doing this only for the famous paintings

In [11]:
pd.set_option('display.max_rows', None)

# GET ONLY highlights
g2 = all[all['isHighlight'] == True]
g2 = g2.groupby(['animal', 'medium']).size().reset_index(name='count').sort_values(by=['animal', 'count'], ascending=[True, False])
# g2["medium"].unique().tolist() #These are all the unique categories

In [12]:
## Define categories - taken GPT help on bucketing due to time constraints!
clamm_categories = {
    'Oil on canvas': 'Oil and other painting',
    'Bronze': 'Metal',
    'Meta-Greywacke': 'Stone',
    'Black stone': 'Stone',
    'Brass; raised, repoussé, inlaid with silver and a black compound': 'Metal',
    'Bronze, ivory': 'Metal',
    'Ebony, ivory': 'Organic',
    'Faience': 'Ceramic',
    'Fresco': 'Oil and other painting',
    'Granodiorite': 'Stone',
    'Hilt: Nephrite\r\nBlade: Watered steel': 'Metal',
    'Ivory': 'Organic',
    'Marble': 'Stone',
    'Marble of different colors, semiprecious stones, Egyptian alabaster, residue of paint of different colors on the piers': 'Stone',
    'Marble, Parian': 'Stone',
    'Meta-greywacke': 'Stone',
    'Oil on canvas, transferred from wood': 'Oil and other painting',
    'Shell (Turbinella pyrum), brass, wax': 'Mixed Media',
    'Travertine (Egyptian alabaster) with traces of gesso and red pigment': 'Stone',
    'Wood, paint, gesso, linen twine and linen fabric': 'Mixed Media',
    'Oil on wood': 'Oil and other painting',
    'Tempera and gold on wood': 'Oil and other painting',
    'Copper alloy': 'Metal',
    'Tempera on wood, gold ground': 'Oil and other painting',
    'Sandstone': 'Stone',
    'Distemper on cloth': 'Oil and other painting',
    'Maple, spruce, ebony': 'Wood',
    'Stone': 'Stone',
    'Acrylic on PVC panels': 'Oil and other painting',
    'Acrylic on canvas filled with foam rubber': 'Oil and other painting',
    'Acrylic on canvas on board': 'Oil and other painting',
    'Amber, gold foil, gilt brass, wood, silk satin, paper': 'Mixed Media',
    'Andesite': 'Stone',
    'Boxwood, rosewood, ebony': 'Wood',
    'Brass; cast and hammered, pierced, chased, inlaid with silver': 'Metal',
    'Brass; cast and turned, engraved, and inlaid with silver, gold, and black organic compound': 'Metal',
    'Bronze alloy': 'Metal',
    'Bronze with traces of gold leaf': 'Metal',
    'Bronze, on a later wood base': 'Metal',
    'Bronze, partially fire-gilt, silver inlay': 'Metal',
    'Bronze, shoe polish, plaster and painted resin': 'Mixed Media',
    'Bronze; cast, chased, and inlaid with copper': 'Metal',
    'Carved walnut and amaranth': 'Wood',
    'Case: partially gilded silver, gilded brass; movement: brass, steel': 'Metal',
    'Champlevé and translucent enamel on copper gilt': 'Metal',
    'Charcoal on paper': 'Drawing',
    'Chrome-plated brass': 'Metal',
    'Copper alloy inlaid with silver and glass or obsidian': 'Metal',
    'Copper alloy with gilding and color': 'Metal',
    'Copper alloy with gilding and semiprecious stones': 'Metal',
    'Cut and pasted printed, colored and metallic papers, photostats, graphite, ink marker, gouache, watercolor, and ink on Masonite': 'Mixed Media',
    'Cypress, boxwood, paint, leather, fir': 'Wood',
    'Earthenware with slip and pigment': 'Ceramic',
    'Earthenware; luster-painted on opaque white glaze': 'Ceramic',
    'Enamel on canvas': 'Oil and other painting',
    'Encaustic, oil, newsprint, and charcoal on canvas': 'Oil and other painting',
    'Faience, bronze or copper alloy, glass, agate, carnelian, lapis lazuli, turquoise': 'Mixed Media',
    'Fresco, mounted on canvas': 'Oil and other painting',
    'Gilt bronze': 'Metal',
    'Gilt bronze with incised decoration': 'Metal',
    'Gilt copper alloy, inlaid with semiprecious stones': 'Metal',
    'Gilt-copper alloy with color and gold paint': 'Metal',
    'Glass, greenish; blown, folded foot; enameled and gilded': 'Glass',
    'Glass, painted and stained': 'Glass',
    'Glass; blown, folded foot, applied handles, enameled, and gilded': 'Glass',
    'Gold': 'Metal',
    'Gold inset with garnet': 'Metal',
    'Gold sheet; worked, chased, and set with turquoise, gray chalcedony, and glass': 'Metal',
    'Gypsum plaster; modeled, carved, polychrome-painted, gilded': 'Mixed Media',
    'Hanging scroll; ink and color on silk': 'Oil and other painting',
    'Hanging scroll; ink on silk with gold on colophon': 'Oil and other painting',
    'High-copper alloy, fire-gilt, brown natural patina where exposed': 'Metal',
    'Hilt: copper; cast, chased, gilded, and inlaid with rubies.\r\nBlade: steel; forged': 'Metal',
    'Ink, graphite, and cut-and-pasted painted and printed papers on paperboard': 'Mixed Media',
    'Ink, opaque watercolor, and gold on paper': 'Oil and other painting',
    'Ivory; carved, inlaid with stone with traces of pigment': 'Organic',
    'Leaded bronze': 'Metal',
    'Limestone': 'Stone',
    'Mahogany (probably Swietenia mahagoni), applied and inlaid silver': 'Wood',
    'Maple, spruce, ebony, mother of pearl': 'Wood',
    'Molded terracotta': 'Ceramic',
    'Oak, cherry, pine, mahogany, veneered with maple, burl woods, holly, hornbeam (all partially stained), tulipwood, mahogany, and other woods; mother-of-pearl; partially gilded and tooled leather; gilt bronze, iron, steel, brass, partially gold-lacquered brass': 'Wood',
    'Oak, pine, walnut, mahogany, and cherry veneered with hornbeam (partially stained), tulipwood, walnut, holly and maple (both partially stained), boxwood, mahogany, and other woods; red brocatelle marble; gilt bronze; iron, steel, and brass': 'Wood',
    'Oil and cut-and-pasted printed paper on wood': 'Mixed Media',
    'Oil and enamel on canvas': 'Oil and other painting',
    'Oil and gold on oak': 'Oil and other painting',
    'Oil and gold on wood': 'Oil and other painting',
    'Oil and tempera on wood': 'Oil and other painting',
    'Oil on Baltic oak': 'Oil and other painting',
    'Oil on Masonite': 'Oil and other painting',
    'Oil on canvas, adhered to cardboard': 'Oil and other painting',
    'Oil on cardboard': 'Oil and other painting',
    'Oil on cardboard laid down on panel.': 'Oil and other painting',
    'Oil on linden': 'Oil and other painting',
    'Oil on oak': 'Oil and other painting',
    'Oil on oak panel': 'Oil and other painting',
    'Oil on panel, transferred to canvas': 'Oil and other painting',
    'Oil on wood; (verso: oil and gold on wood)': 'Oil and other painting',
    'Oil on wood; verso: oil and gold on wood': 'Oil and other painting',
    'Oil, gold, and white metal on spruce': 'Oil and other painting',
    'Oil, graphite, ink, and gold leaf on paperboard (Upson board)': 'Mixed Media',
    'Pastel on light green wove paper, now discolored to warm gray, affixed to original pulpboard mount': 'Drawing',
    'Porcelain with peach-bloom glaze (Jingdezhen ware)': 'Ceramic',
    'Red clay': 'Ceramic',
    'Schist': 'Stone',
    'Set of five handscrolls; ink, color, and cut gold leaf (kirikane) on paper': 'Oil and other painting',
    'Silver and ebony': 'Mixed Media',
    'Silver, partly gilt; niello, jewels': 'Metal',
    'Single-channel video, color, sound, 19 min.': 'Mixed Media',
    'Soft-paste porcelain decorated in polychrome enamels, gold': 'Ceramic',
    'Spruce, bone, parchment, snakewood, ivory': 'Mixed Media',
    'Spruce, ebony, fruitwood, bone, ivory, mother-of-pearl, parchment': 'Mixed Media',
    'Spruce, ebony, ivory, tortoiseshell, mother-of-pearl': 'Mixed Media',
    'Spruce, ebony, ivory, tortoiseshell, mother-of-pearl, brass': 'Mixed Media',
    'Spruce, maple, ebony, paint, gilding': 'Mixed Media',
    'Spruce, rosewood, cedar, ebony, ivory or bone': 'Wood',
    'Steel, aluminum, rubber': 'Metal',
    'Steel, gold, copper alloy, leather': 'Metal',
    'Steel, silver, gilt copper, gilt brass, basswood; replaced mirror glass': 'Metal',
    'Stonepaste; openwork, underglaze-painted, glazed in transparent turquoise': 'Ceramic',
    'Stoneware with carved and incised decoration under celadon glaze (Yue ware)': 'Ceramic',
    'Stucco': 'Other',
    'Stucco with traces of paint': 'Other',
    'Tempera and gold leaf on wood': 'Other',
    'Tempera and oil transferred to Masonite': 'Oil and other painting',
    'Tempera on canvas, transferred from wood': 'Other',
    'Tempera on wood': 'Other',
    'Tempera on wood, gold ground, and silver': 'Other',
    'Tempera, oil, and gold on wood': 'Oil and other painting',
    'Tempera, silver, and gold on wood': 'Other',
    'Ten panels: Egg tempera with oil glazing over Permalba on a gesso ground on linen mounted to wood panels with a honeycomb interior': 'Oil and other painting',
    'Terracotta': 'Ceramic',
    'Travertine (Egyptian alabaster), copper, quartz crystal, paint': 'Stone',
    'Walnut veneered with ebony, marquetry of engraved brass and tortoiseshell, gilt-bronze mounts, verd antique marble': 'Wood',
    'Watercolor and gouache on paper': 'Drawing',
    'Wood, Naugahyde': 'Wood',
    'Wood, concrete and steel': 'Wood',
    'Wood, painted and gilt': 'Wood',
    'Wool, silk, silver-gilt thread.': 'Organic',
    'Zinc alloy; cast, engraved, inlaid with brass (bidri ware)': 'Metal',
    'Ebony, beech, brass, leather, paper, parchment, ivory': 'Wood',
    'Elephant ivory': 'Organic',
    'Free-blown glass': 'Glass',
    'Grisaille, tempera, and ink on vellum': 'Drawing',
    'Image: Ink, opaque watercolor, and gold on paper\r\nMargins: Gold on dyed paper': 'Drawing',
    'Ink and gold on paper; leather binding': 'Drawing',
    'Ivory (elephant)': 'Organic',
    'Limestone, ink': 'Stone',
    'Main support: ink, opaque watercolor, and gold on paper; binding: leather': 'Drawing',
    'Marble (Lunense marble from Carrara)': 'Stone',
    'Paper (four layers of pasteboard) with pen and ink, opaque paint, glazes, and applied silver and gold': 'Drawing',
    'Satinwood veneer, oak, spruce, iron, steel, ebony, ivory, gilding, mother-of-pearl, holly, mahogany, burl walnut, tulipwood, silver wire': 'Wood',
    'Silver, silver gilt, and painted enamel': 'Metal',
    'Steel, gold, wood (cherry), staghorn': 'Metal',
    'Wool warp with wool, silk, silver, and gilt wefts': 'Organic',
    'Wool, silk (14 warps per inch; 5-6 per cm)': 'Organic',
    'Wool, silk, and gilt-metal-strip-wrapped silk in slit, dovetailed, and double interlocking tapestry weave.': 'Organic',
    'Distemper on cotton': 'Organic',
    'Brass': 'Metal',
    'Elephant ivory, polychromy, and gilding': 'Organic',
    'Elephant ivory, traces of gilding': 'Organic',
    'Elephant ivory, traces of polychromy': 'Organic',
    'Elephant ivory, whale bone, traces of paint, and gilding': 'Organic',
    'Elephant ivory, with metal mounts': 'Organic',
    'Gabbro; carved': 'Stone',
    'Ivory (elephant), wood, blue-pigment inlay': 'Organic',
    'Ivory, iron, copper (?)': 'Organic',
    'Ivory, wood, polychrome': 'Organic',
    'Stoneware with underglaze cobalt-blue decoration': 'Ceramic',
    'Wood, ivory, parchment, metal': 'Wood',
    'Bronze, silver': 'Metal',
    'Case and dial of enameled gold; hand of steel; movement of brass, partly gilded, and steel': 'Metal',
    'Copper alloy, enamel, gold': 'Metal',
    'Granite': 'Stone',
    'Ink, opaque watercolor, gold, and silver on paper': 'Drawing',
    'Iron, lacquer, copper-gold alloy (shakudō), silver, silk, horse hair, ivory': 'Metal',
    'Ivory, garnet, paint': 'Organic',
    'Limestone, paint': 'Stone',
    'Marble, Hymettian': 'Stone',
    'Oil on linden': 'Oil and other painting',
    'Oil on oak': 'Oil and other painting',
    'Oil on oak panel': 'Oil and other painting',
    'Oil on panel, transferred to canvas': 'Oil and other painting',
    'Oil on wood; (verso: oil and gold on wood)': 'Oil and other painting',
    'Oil on wood; verso: oil and gold on wood': 'Oil and other painting',
    'Opaque watercolor, ink, silver, and gold on paper': 'Drawing',
    'Painted enamel on copper, partly gilt.': 'Metal',
    'Partially tinted bronze, cotton tarlatan, silk satin, and wood': 'Metal',
    'Pencil, ink, and watercolor on muslin': 'Drawing',
    'Pot-metal glass, vitreous paint': 'Glass',
    'Pot-metal glass, vitreous paint, and lead': 'Glass',
    'Sandstone, paint': 'Stone',
    'Silk, metal wrapped thread; lampas': 'Organic',
    'Steel, copper alloy, gold': 'Metal',
    'Steel, gold, leather, textile': 'Metal',
    'Steel, gold, silver': 'Metal',
    'Steel, gold, silver, leather, textile': 'Metal',
    'Stonepaste; luster-painted on opaque monochrome glaze': 'Ceramic',
    'Tin-glazed earthenware': 'Ceramic',
    'Wood (poplar) with gesso relief, gold and tin leaf, glazes and paint; wood (cypress, poplar, and mulberry), mother-of-pearl, marble and other stones, stucco with glass, plaster ceramic tiles, iron, brass': 'Wood',
    'Wood (teak); carved': 'Wood',
    'Wood, Chinese trade ceramic, tin (the protrusion/plug in the vessel)': 'Wood',
    'Wood, pigment': 'Wood',
    'Yellow pine, tulip poplar': 'Wood',
    'Brass; cast, chased, and inlaid with silver and black compound': 'Metal',
    'Bronze; cast, engraved, chased, pierced': 'Metal',
    'Copper: engraved and gilt; champlevé enamel: blue-black, medium blue, turquoise, green, red, and white': 'Metal',
    'Cotton (warp and weft), wool (pile); asymmetrically knotted pile': 'Organic',
    'Egyptian blue, gold': 'Stone',
    'Faience (tin-glazed earthenware)': 'Ceramic',
    'Faience, remains of a travertine (Egyptian alabaster) tenon': 'Ceramic',
    'Free-blown glass with paint and metallic foils': 'Glass',
    'Gilt brass with silver and copper inlay': 'Metal',
    'Glassy faience': 'Ceramic',
    'Gneiss': 'Stone',
    'Gold, glass, rock crystal, agate, carnelian': 'Metal',
    'Gold, rock crystal, emerald': 'Metal',
    'Granite, paint': 'Stone',
    'Granulite': 'Stone',
    'Hard-paste porcelain': 'Ceramic',
    'Limestone, traces of paint': 'Stone',
    'Marble (Carrara marble)': 'Stone',
    'Marble, Parian 2': 'Stone',
    'Oak veneered with tulipwood, amaranth, holly, ebonized holly, and other marquetry wood; brass; green-colored metal; gilt-bronze mounts; marble top.  Soft paste porcelain plaques from the Sèvres Manufactory.': 'Wood',
    'Porphyry': 'Stone',
    'Quartzite': 'Stone',
    'Rosewood, brass, mother-of-pearl, and abalone': 'Wood',
    'Silk (warp and weft), wool (pile); asymmetrically knotted pile': 'Organic',
    'Silver': 'Metal',
    'Silver, enamel, steel, paste brilliants, wood, fish skin': 'Metal',
    'Slip-decorated earthenware': 'Ceramic',
    'Steel, copper, gold, glass, pigment, textile': 'Metal',
    'Steel, gold, brass, silk, cotton, metallic yarn, paper': 'Metal', 
    'Steel, silver, ivory, wood, leather': 'Metal',
    'Tempera, gold, and ink on vellum': 'Drawing',
    'Various woods, ivory and tortoiseshell inlays, gold and silver inlays, metalwork, cloth, laquer, paper,': 'Mixed Media',
    'Walrus ivory': 'Organic',
    'Wood (ficus sycomorus?)': 'Wood',
    'Alkaline-glazed stoneware with kaolin': 'Ceramic',
    'Silver, silver-gilt': 'Metal',
    'colored wax, paint, glass': 'Mixed Media'}

# Function to classify medium using CLaMM categories
def classify_medium(medium):
    return clamm_categories.get(medium, 'Other')

# Apply classification to each row
classified_data = []
for index, row in enumerate(g2['medium']):
    classified_data.append((g2['animal'][index], row, g2['count'][index], classify_medium(row)))

# Display classified data
classified_df = pd.DataFrame(classified_data, columns=['animal', 'medium', 'count', 'clamm_category'])
# classified_df

In [13]:
chart3 = classified_df.groupby(['animal', 'clamm_category']).size().reset_index(name='count').sort_values(by=['animal', 'count'], ascending=[True, False])

In [14]:
chart3 = chart3.sort_values(by='animal', ascending = False)
chart3

## Custom order for chart
# chart3["clamm_category"].unique().tolist()

custom_order = [
    'Stone',
    'Metal',
    'Oil and other painting',
    'Mixed Media',
    'Organic',
    'Ceramic',
    'Wood',
    'Glass',
    'Drawing',
    'Other'
]

# Create a categorical dtype with custom order
cat_type = pd.CategoricalDtype(categories=custom_order, ordered=True)

# Sort the DataFrame by 'animal' and 'category' with custom ordering
sorted_df = chart3.sort_values(by=['animal', 'clamm_category'], key=lambda x: x.astype(cat_type))

#sorted_df.to_csv("./DATA/CHARTS/chart3.csv", index = False)

## 4. Chart 4: GEOGRAPHY - Where do all the CAT paintings come from?

In [16]:
# Group by countries, clean up manually for old countries + remove "probably". Smallish dataset, easily done by hand.
geog = all.groupby(['animal', 'country']).size().reset_index(name='count').sort_values(by=['animal', 'count'], ascending=[True, False])
geog_cats = geog[geog["animal"] == "cat"]
# geog_cats

# geog_cats.to_csv("./DATA/CHARTS/chart4.csv", index = False)

## ENDS! 🐈 🐈 🐈 🐈 🐈